In [1]:
import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
# pretrained_model_path = "ds/models/first_stage_2462.pth"
# weights = torch.load(pretrained_model_path)

In [4]:
e = env_module.Environment()
e.reset()

In [5]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, 77)).clone()
obs_ = torch.cat([obs, obs], dim=0)
print(f"size: {obs_.size()}, type: {obs_.dtype}")

size: torch.Size([2, 77]), type: torch.int64


In [6]:
e.obs_list

[[{'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60,
    'step': 0,
    'geese': [[57], [49], [74], [64]],
    'food': [50, 11],
    'index': 0},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 1},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 2},
   'status': 'ACTIVE'},
  {'action': 'NORTH',
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60, 'index': 3},
   'status': 'ACTIVE'}]]

In [7]:
input_ = torch.randn(2, 77)
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 77]), type: torch.float32


In [8]:
net = env_module.GeeseNetGTrXL(e)

In [9]:
# net.geese_net.load_state_dict(weights, strict=False)

In [10]:
# Disable trainable for pretrained weights.
# for param in net.geese_net.parameters():
#     param.requires_grad = False
# for param in net.geese_net.conv_p.parameters():
#     param.requires_grad = True
# for param in net.geese_net.conv_v.parameters():
#     param.requires_grad = True

In [11]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

6,781,329


In [12]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

6,781,329


In [13]:
net

GeeseNetGTrXL(
  (encoder): GeeseEncoder(
    (embed): Embedding(18, 8)
  )
  (gtrxl): GTrXL(
    (embed): PositionalEncoding()
    (transfomer): TransformerEncoder(
      (layers): ModuleList(
        (0): TEL(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=616, out_features=616, bias=True)
          )
          (linear1): Linear(in_features=616, out_features=256, bias=True)
          (dropout): Dropout(p=0, inplace=False)
          (linear2): Linear(in_features=256, out_features=616, bias=True)
          (norm1): LayerNorm((616,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((616,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0, inplace=False)
          (dropout2): Dropout(p=0, inplace=False)
          (gru_1): GRU(616, 616, batch_first=True)
          (gru_2): GRU(616, 616, batch_first=True)
        )
      )
    )
  )
  (head): GeeseHead(
    (head_p_1): Linear(in_features=616, out_features=

In [14]:
out = net(obs_)
print(f"size: {out['policy'].size()}, type: {out['policy'].dtype}")

size: torch.Size([2, 4]), type: torch.float32


In [15]:
out

{'policy': tensor([[-0.1927, -0.0793, -0.1033,  0.0715],
         [-0.1192, -0.0534, -0.0562,  0.0184]], grad_fn=<MmBackward>),
 'value': tensor([[-0.0725],
         [-0.0881]], grad_fn=<TanhBackward>)}